In [3]:
from datasets.get_datasets import *
from revision import *
from boostsrl import boostsrl
import collections
import re
import pandas as pd

In [19]:
[facts, pos, neg] = get_imdb_dataset('workedunder', folds=True)
table = []
for fold in range(len(facts)):
    facts[fold] += pos[fold]
    for i in range(len(facts[fold])):
        facts[fold][i] = facts[fold][i].split('(')[0]
    t = []
    columns = []
    d = dict(collections.Counter(facts[fold]))
    for key, value in sorted(d.items()):
        t.append(value)
        columns.append(key)
    table.append(t)

pd.DataFrame(table, columns=columns)

Counter({'movie': 64, 'workedunder': 56, 'actor': 55, 'female': 26, 'genre': 11, 'director': 3})
Counter({'movie': 62, 'workedunder': 58, 'actor': 55, 'female': 17, 'genre': 12, 'director': 4})
Counter({'workedunder': 178, 'movie': 62, 'actor': 43, 'female': 22, 'genre': 18, 'director': 18})
Counter({'movie': 49, 'workedunder': 45, 'actor': 40, 'female': 9, 'genre': 4, 'director': 4})
Counter({'movie': 49, 'workedunder': 45, 'actor': 43, 'female': 21, 'director': 3, 'genre': 2})


,actor,director,female,genre,movie,workedunder
0,55,3,26,11,64,56
1,55,4,17,12,62,58
2,43,18,22,18,62,178
3,40,4,9,4,49,45
4,43,3,21,2,49,45


In [20]:
[facts, pos, neg] = get_uwcse_dataset('advisedby', folds=True, acceptedPredicates=[
'professor',
'student',
'advisedby',
'tempadvisedby',
'ta',
'hasposition',
'publication',
'inphase',
'courselevel',
'yearsinprogram',
'projectmember',
])
table = []
for fold in range(len(facts)):
    facts[fold] += pos[fold]
    for i in range(len(facts[fold])):
        facts[fold][i] = facts[fold][i].split('(')[0]
    t = []
    columns = []
    d = dict(collections.Counter(facts[fold]))
    for key, value in sorted(d.items()):
        t.append(value)
        columns.append(key)
    table.append(t)

pd.DataFrame(table, columns=columns)

,advisedby,courselevel,hasposition,inphase,professor,projectmember,publication,student,ta,tempadvisedby,yearsinprogram
0,35,30,14,47,14,1,292,54,20,11,47.0
1,20,29,9,22,9,73,52,43,7,22,NaN
2,9,14,4,10,8,10,20,26,1,10,NaN
3,33,31,16,40,18,247,54,53,10,40,NaN
4,16,28,9,21,13,4,112,36,53,8,21.0


In [23]:
[facts, pos, neg] = get_nell_dataset('athleteplaysforteam')

table = []
facts += pos
for i in range(len(facts)):
    facts[i] = facts[i].split('(')[0]
t = []
columns = []
d = dict(collections.Counter(facts))
for key, value in sorted(d.items()):
    t.append(value)
    columns.append(key)
table.append(t)

pd.DataFrame(table, columns=columns)

,athleteledsportsteam,athleteplaysforteam,athleteplaysinleague,athleteplayssport,teamalsoknownas,teamplaysagainstteam,teamplaysinleague,teamplayssport
0,421,1458,1005,1718,316,2874,1502,397


## Test Finding connected components

In [24]:
facts = ['relationa(a,b)',
        'relationb(c,a)',
        'relationc(b,a)',
        'relationc(b,d)',
        'relationa(e,f)',
        'relationb(f,g)',
        'relationd(h,h)',
        'relationd(h,i)',
        'relationa(i,j)',
        'relationb(j,k)',
        'relationa(a,k)']
sports = set()
teams = set()
leagues = set()
relations = set()
entities = set()
graphs = []
relations_in_graphs = []
facts_in_graphs = []
for fact in facts:
    match = re.findall('(\w*)\s*\(\s*(\w*)[,\s]*(\w*)\s*\)', fact)
    if match:
        relation = match[0][0]
        subjec = match[0][1]
        objec = match[0][2]
        relations.add(relation)
        entities.add(subjec)
        entities.add(objec)
        if relation in ['athleteplayssport', 'teamplayssport']:
            sports.add(objec)
        if relation in ['athleteledsportsteam', 'athleteplaysforteam']:
            teams.add(objec)
        if relation in ['teamalsoknownas', 'teamplaysagainstteam']:
            teams.add(subjec)
            teams.add(objec)
        if relation in ['athleteplaysinleague', 'teamplaysinleague']:
            leagues.add(objec)
        print(fact)
        left_found = None
        right_found = None
        for i in range(len(graphs)):
            if subjec in graphs[i]:
                left_found = i
                break
        for i in range(len(graphs)):
            if objec in graphs[i]:
                right_found = i
                break
        if left_found != None and right_found != None and left_found == right_found:
            relations_in_graphs[left_found].add(relation)
            relations_in_graphs[left_found] = relations_in_graphs[left_found].union(relations_in_graphs[right_found])
            facts_in_graphs[left_found].add(fact)
            facts_in_graphs[left_found] = facts_in_graphs[left_found].union(facts_in_graphs[right_found])
        elif left_found != None and right_found != None and left_found != right_found:
            graphs[left_found] = graphs[left_found].union(graphs[right_found])
            relations_in_graphs[left_found].add(relation)
            relations_in_graphs[left_found] = relations_in_graphs[left_found].union(relations_in_graphs[right_found])
            facts_in_graphs[left_found].add(fact)
            facts_in_graphs[left_found] = facts_in_graphs[left_found].union(facts_in_graphs[right_found])
            del graphs[right_found]
            del relations_in_graphs[right_found]
            del facts_in_graphs[right_found]
        elif left_found != None and right_found == None:
            graphs[left_found].add(objec)
            relations_in_graphs[left_found].add(relation)
            facts_in_graphs[left_found].add(fact)
        elif left_found == None and right_found != None:
            graphs[right_found].add(subjec)
            relations_in_graphs[right_found].add(relation)
            facts_in_graphs[right_found].add(fact)
        elif left_found == None and right_found == None:
            graphs.append(set([subjec, objec]))
            relations_in_graphs.append(set([relation]))
            facts_in_graphs.append(set([fact]))
    print(graphs)
print('Relations: %s' % relations)
print('Entities: %s' % entities)
print('Graphs: %s' % graphs)
print('Relations in graph: %s' % relations_in_graphs)
print('Facts in graph: %s' % facts_in_graphs)

relationa(a,b)
[{'b', 'a'}]
relationb(c,a)
[{'b', 'c', 'a'}]
relationc(b,a)
[{'b', 'c', 'a'}]
relationc(b,d)
[{'b', 'd', 'c', 'a'}]
relationa(e,f)
[{'b', 'd', 'c', 'a'}, {'f', 'e'}]
relationb(f,g)
[{'b', 'd', 'c', 'a'}, {'f', 'g', 'e'}]
relationd(h,h)
[{'b', 'd', 'c', 'a'}, {'f', 'g', 'e'}, {'h'}]
relationd(h,i)
[{'b', 'd', 'c', 'a'}, {'f', 'g', 'e'}, {'i', 'h'}]
relationa(i,j)
[{'b', 'd', 'c', 'a'}, {'f', 'g', 'e'}, {'j', 'i', 'h'}]
relationb(j,k)
[{'b', 'd', 'c', 'a'}, {'f', 'g', 'e'}, {'j', 'i', 'h', 'k'}]
relationa(a,k)
[{'b', 'k', 'c', 'a', 'h', 'j', 'i', 'd'}, {'f', 'g', 'e'}]
Relations: {'relationa', 'relationd', 'relationc', 'relationb'}
Entities: {'b', 'k', 'f', 'c', 'a', 'h', 'j', 'i', 'd', 'g', 'e'}
Graphs: [{'b', 'k', 'c', 'a', 'h', 'j', 'i', 'd'}, {'f', 'g', 'e'}]
Relations in graph: [{'relationa', 'relationd', 'relationc', 'relationb'}, {'relationa', 'relationb'}]
Facts in graph: [{'relationa(i,j)', 'relationd(h,h)', 'relationd(h,i)', 'relationa(a,b)', 'relationc(b,d)', '

## Finding connected components

In [25]:
[facts, pos, neg] = get_nell_dataset('athleteplaysforteam')
facts += pos
sports = set()
teams = set()
leagues = set()
relations = set()
entities = set()
graphs = []
relations_in_graphs = []
facts_in_graphs = []
for fact in facts:
    match = re.findall('(\w*)\s*\(\s*(\w*)[,\s]*(\w*)\s*\)', fact)
    if match:
        relation = match[0][0]
        subjec = match[0][1]
        objec = match[0][2]
        relations.add(relation)
        entities.add(subjec)
        entities.add(objec)
        if relation in ['athleteplayssport', 'teamplayssport']:
            sports.add(objec)
        if relation in ['athleteledsportsteam', 'athleteplaysforteam']:
            teams.add(objec)
        if relation in ['teamalsoknownas', 'teamplaysagainstteam']:
            teams.add(subjec)
            teams.add(objec)
        if relation in ['athleteplaysinleague', 'teamplaysinleague']:
            leagues.add(objec)
        left_found = None
        right_found = None
        for i in range(len(graphs)):
            if subjec in graphs[i]:
                left_found = i
                break
        for i in range(len(graphs)):
            if objec in graphs[i]:
                right_found = i
                break
        if left_found != None and right_found != None and left_found == right_found:
            relations_in_graphs[left_found].add(relation)
            relations_in_graphs[left_found] = relations_in_graphs[left_found].union(relations_in_graphs[right_found])
            facts_in_graphs[left_found].add(fact)
            facts_in_graphs[left_found] = facts_in_graphs[left_found].union(facts_in_graphs[right_found])
        elif left_found != None and right_found != None and left_found != right_found:
            graphs[left_found] = graphs[left_found].union(graphs[right_found])
            relations_in_graphs[left_found].add(relation)
            relations_in_graphs[left_found] = relations_in_graphs[left_found].union(relations_in_graphs[right_found])
            facts_in_graphs[left_found].add(fact)
            facts_in_graphs[left_found] = facts_in_graphs[left_found].union(facts_in_graphs[right_found])
            del graphs[right_found]
            del relations_in_graphs[right_found]
            del facts_in_graphs[right_found]
        elif left_found != None and right_found == None:
            graphs[left_found].add(objec)
            relations_in_graphs[left_found].add(relation)
            facts_in_graphs[left_found].add(fact)
        elif left_found == None and right_found != None:
            graphs[right_found].add(subjec)
            relations_in_graphs[right_found].add(relation)
            facts_in_graphs[right_found].add(fact)
        elif left_found == None and right_found == None:
            graphs.append(set([subjec, objec]))
            relations_in_graphs.append(set([relation]))
            facts_in_graphs.append(set([fact]))

In [26]:
relations

{'athleteledsportsteam',
 'athleteplaysforteam',
 'athleteplaysinleague',
 'athleteplayssport',
 'teamalsoknownas',
 'teamplaysagainstteam',
 'teamplaysinleague',
 'teamplayssport'}

In [27]:
sports

{'baseball',
 'basketball',
 'boxing',
 'carracing',
 'collegebaseball',
 'cycling',
 'football',
 'golf',
 'golfing',
 'hockey',
 'lacrosse',
 'professionalbasketball',
 'professionalfootball',
 'sailing',
 'soccer',
 'softball',
 'swimming',
 'tennis',
 'trackandfield',
 'wrestling'}

In [28]:
teams

{'acmilan',
 'akron',
 'alabamacrimsontide',
 'alabamastateuniversity',
 'anaheimangels',
 'anaheimducks',
 'antiochcollege',
 'aquinascollege',
 'arizonacardinals',
 'arizonadiamondbacks',
 'arizonastateuniversity',
 'arkansasrazorbacks',
 'arkansasstateuniversity',
 'armyblackknightsfootball',
 'ashlanduniversity',
 'astroslastnight',
 'atlantabraves',
 'atlantabravesspringtraining',
 'atlantathrashers',
 'auburnuniversity',
 'avs',
 'ballstateuniversity',
 'baltimorecolts',
 'baltimorecoltsinsuperbowliii',
 'baltimoreorioles',
 'barcelona',
 'barcelonadragons',
 'baylorbears',
 'baylorbearsbasketball',
 'bayloruniversity',
 'baylorwomen',
 'bclions',
 'bears',
 'bemidjistatebeavers',
 'benfica',
 'bengals',
 'bills',
 'blackhawks',
 'bluejackets',
 'bluejays',
 'boisest',
 'boisestateuniversity',
 'boltonwanderers',
 'bostonbraves',
 'bostonbruins',
 'bostonceltics',
 'bostoncollege',
 'bostonredsox',
 'bostonuniversity',
 'boulder',
 'bowlinggreenstate',
 'bowlinggreenstateuniversi

In [29]:
leagues

{'acc',
 'afc',
 'bcs',
 'cfl',
 'fa',
 'formulaone',
 'international',
 'majorleaguebaseball',
 'mlb',
 'mls',
 'nascar',
 'nba',
 'ncaa',
 'nfl',
 'nhl',
 'sec',
 'uefa',
 'wnba'}

In [30]:
s = [len(graph) for graph in graphs]
t = [len(graph) for graph in facts_in_graphs]
print('Total entities: %s' % sum(s))
print('Total entities: %s' % len(entities))
print('Total facts: %s' % len(facts))
print('Total facts: %s' % sum(t))

Total entities: 4538
Total entities: 4538
Total facts: 9691
Total facts: 9629


In [31]:
facts_in_graphs

[{'athleteplayssport(player,golfing).'},
 {'athleteplayssport(andrewgolota,boxing).',
  'athleteplayssport(antoniomargarito,boxing).',
  'athleteplayssport(antoniotarver,boxing).',
  'athleteplayssport(bernardhopkins,boxing).',
  'athleteplayssport(chrisbyrd,boxing).',
  'athleteplayssport(diegocorrales,boxing).',
  'athleteplayssport(erikmorales,boxing).',
  'athleteplayssport(evanderholyfield,boxing).',
  'athleteplayssport(floydmayweather,boxing).',
  'athleteplayssport(floydmayweatherjr,boxing).',
  'athleteplayssport(georgeforeman,boxing).',
  'athleteplayssport(gerrycooney,boxing).',
  'athleteplayssport(hasimrahman,boxing).',
  'athleteplayssport(jefflacy,boxing).',
  'athleteplayssport(jermaintaylor,boxing).',
  'athleteplayssport(joecalzaghe,boxing).',
  'athleteplayssport(joefrazier,boxing).',
  'athleteplayssport(johnduddy,boxing).',
  'athleteplayssport(johnruiz,boxing).',
  'athleteplayssport(joseluiscastillo,boxing).',
  'athleteplayssport(juandiaz,boxing).',
  'athletepl

In [32]:
for i in range(len(graphs)):
    total = len(graphs[i])
    sp = set()
    for sport in sports:
        if sport in graphs[i]:
            sp.add(sport)
    print('Graph: %s, Total entities: %s, Sports in graph: %s' % (i+1, total, sp))

Graph: 1, Total entities: 2, Sports in graph: {'golfing'}
Graph: 2, Total entities: 37, Sports in graph: {'boxing'}
Graph: 3, Total entities: 33, Sports in graph: {'carracing'}
Graph: 4, Total entities: 7, Sports in graph: set()
Graph: 5, Total entities: 3, Sports in graph: set()
Graph: 6, Total entities: 4405, Sports in graph: {'hockey', 'collegebaseball', 'softball', 'golf', 'baseball', 'tennis', 'soccer', 'football', 'basketball'}
Graph: 7, Total entities: 2, Sports in graph: {'lacrosse'}
Graph: 8, Total entities: 23, Sports in graph: {'wrestling'}
Graph: 9, Total entities: 2, Sports in graph: {'professionalbasketball'}
Graph: 10, Total entities: 3, Sports in graph: {'trackandfield'}
Graph: 11, Total entities: 2, Sports in graph: {'sailing'}
Graph: 12, Total entities: 2, Sports in graph: {'swimming'}
Graph: 13, Total entities: 2, Sports in graph: set()
Graph: 14, Total entities: 2, Sports in graph: {'professionalfootball'}
Graph: 15, Total entities: 2, Sports in graph: {'cycling'}
G

In [33]:
for i in range(len(graphs)):
    total = len(graphs[i])
    sp = set()
    for league in leagues:
        if league in graphs[i]:
            sp.add(league)
    print('Graph: %s, Total entities: %s, Leagues in graph: %s' % (i+1, total, sp))

Graph: 1, Total entities: 2, Leagues in graph: set()
Graph: 2, Total entities: 37, Leagues in graph: set()
Graph: 3, Total entities: 33, Leagues in graph: {'nascar'}
Graph: 4, Total entities: 7, Leagues in graph: {'wnba'}
Graph: 5, Total entities: 3, Leagues in graph: {'cfl'}
Graph: 6, Total entities: 4405, Leagues in graph: {'nfl', 'afc', 'international', 'sec', 'majorleaguebaseball', 'ncaa', 'bcs', 'acc', 'mlb', 'fa', 'mls', 'nhl', 'uefa', 'nba'}
Graph: 7, Total entities: 2, Leagues in graph: set()
Graph: 8, Total entities: 23, Leagues in graph: set()
Graph: 9, Total entities: 2, Leagues in graph: set()
Graph: 10, Total entities: 3, Leagues in graph: set()
Graph: 11, Total entities: 2, Leagues in graph: set()
Graph: 12, Total entities: 2, Leagues in graph: set()
Graph: 13, Total entities: 2, Leagues in graph: {'formulaone'}
Graph: 14, Total entities: 2, Leagues in graph: set()
Graph: 15, Total entities: 2, Leagues in graph: set()
Graph: 16, Total entities: 2, Leagues in graph: set()


In [34]:
for i in range(len(graphs)):
    total = len(graphs[i])
    sp = set()
    for team in teams:
        if team in graphs[i]:
            sp.add(team)
    print('Graph: %s, Total entities: %s, Teams in graph: %s' % (i+1, total, sp))

Graph: 1, Total entities: 2, Teams in graph: set()
Graph: 2, Total entities: 37, Teams in graph: set()
Graph: 3, Total entities: 33, Teams in graph: {'bradkeselowkski', 'trevorbayne'}
Graph: 4, Total entities: 7, Teams in graph: {'houstoncomets'}
Graph: 5, Total entities: 3, Teams in graph: {'bclions', 'edmontoneskimos'}
Graph: 6, Total entities: 4405, Teams in graph: {'harvard', 'losangelesangelsofanaheim', 'oregonstateuniversity', 'californiainstituteoftechnology', 'ncaayouthkids', 'sandiegostateuniversity', 'jayhawks', 'floridagatorsbasketball', 'iowastatecyclones', 'arizonastateuniversity', 'clevelandindians', 'pittsburgstateuniversity', 'kentuckywildcatsbasketball', 'usdaars', 'kansasuniversitymedicalschool', 'kansasstateuniversity', 'torontomapleleafs', 'minnesotastateuniversitymankato', 'timberwolves', 'baylorwomen', 'youngstownstateuniversity', 'baylorbears', 'sixers', 'redsox', 'eastmanschoolofmusic', 'carolina', 'washingtonstate', 'longbeachstate', 'ballstateuniversity', 'rad

In [35]:
for i in range(len(graphs)):
    total = len(graphs[i])
    sp = relations_in_graphs[i]
    print('Graph: %s, Total entities: %s, Relations in graph: %s' % (i+1, total, sp))

Graph: 1, Total entities: 2, Relations in graph: {'athleteplayssport'}
Graph: 2, Total entities: 37, Relations in graph: {'athleteplayssport'}
Graph: 3, Total entities: 33, Relations in graph: {'teamalsoknownas', 'athleteplaysinleague', 'athleteplaysforteam', 'teamplaysinleague', 'teamplaysagainstteam', 'athleteplayssport', 'athleteledsportsteam'}
Graph: 4, Total entities: 7, Relations in graph: {'teamplaysinleague', 'athleteplaysforteam'}
Graph: 5, Total entities: 3, Relations in graph: {'teamalsoknownas', 'teamplaysinleague'}
Graph: 6, Total entities: 4405, Relations in graph: {'athleteplaysinleague', 'teamplayssport', 'teamplaysinleague', 'teamplaysagainstteam', 'athleteplayssport', 'athleteplaysforteam', 'teamalsoknownas', 'athleteledsportsteam'}
Graph: 7, Total entities: 2, Relations in graph: {'teamplayssport'}
Graph: 8, Total entities: 23, Relations in graph: {'athleteplayssport', 'teamplayssport'}
Graph: 9, Total entities: 2, Relations in graph: {'athleteplayssport'}
Graph: 10,

In [36]:
for i in range(len(graphs)):
    total = len(graphs[i])
    sp = facts_in_graphs[i] if total < 40 else set()
    print('Graph: %s, Total entities: %s, Facts in graph: \n%s' % (i+1, total, '\n'.join(list(sp))))
    print('\n')

Graph: 1, Total entities: 2, Facts in graph: 
athleteplayssport(player,golfing).


Graph: 2, Total entities: 37, Facts in graph: 
athleteplayssport(evanderholyfield,boxing).
athleteplayssport(mannypacquiao,boxing).
athleteplayssport(wladimirklitschko,boxing).
athleteplayssport(juanmanuelmarquez,boxing).
athleteplayssport(miketyson,boxing).
athleteplayssport(antoniomargarito,boxing).
athleteplayssport(robertoduran,boxing).
athleteplayssport(lamontpeterson,boxing).
athleteplayssport(oscardelahoya,boxing).
athleteplayssport(vitaliklitschko,boxing).
athleteplayssport(joecalzaghe,boxing).
athleteplayssport(jermaintaylor,boxing).
athleteplayssport(chrisbyrd,boxing).
athleteplayssport(mikkelkessler,boxing).
athleteplayssport(floydmayweatherjr,boxing).
athleteplayssport(juandiaz,boxing).
athleteplayssport(joseluiscastillo,boxing).
athleteplayssport(diegocorrales,boxing).
athleteplayssport(kellypavlik,boxing).
athleteplayssport(gerrycooney,boxing).
athleteplayssport(erikmorales,boxing).
athlete

In [42]:
#import json
#with open('nell_graph6.json', 'w') as outfile:
#    json.dump(list(facts_in_graphs[5]), outfile)

In [38]:
""" Graph Class
"""

class Node(object):
    def __init__(self, name):
        self.name = name
        self.relations = []
        self.visited = False
        
    def add_relation(self, rel, obj):
        self.relations.append((rel, obj))
        obj.relations.append(('_' + rel, self))
        
class Graph(object):
    def __init__(self):
        self.nodes = {}
        
    def add_relation(self, sub, rel, obj):
        if sub not in self.nodes:
            self.nodes[sub] = Node(sub)
        if obj not in self.nodes:
            self.nodes[obj] = Node(obj)
        self.nodes[sub].add_relation(rel, self.nodes[obj])

def get_subgraph(root, depth, max_depth):
    facts = set()
    root.visited = True
    for relation in root.relations:
        if relation[0][0] == '_':
            s = relation[0][1:] + '(' + relation[1].name + ',' + root.name + ').'
        else:
            s = relation[0] + '(' + root.name + ',' + relation[1].name + ').'
        facts.add(s)
        if relation[1].visited == False and depth < max_depth:
            subs = get_subgraph(relation[1], depth+1, max_depth)
            facts = facts.union(subs)
    return facts

In [41]:
import re     

facts_sports = {}
for sport in ['hockey', 'basketball', 'baseball', 'tennis', 'golf', 'soccer', 'softball', 'football', 'collegebaseball']:
    g = Graph()
    for fact in facts:
        match = re.findall('(\w*)\s*\(\s*(\w*)[,\s]*(\w*)\s*\)', fact)
        if match:
            relation = match[0][0]
            subjec = match[0][1]
            objec = match[0][2]
            g.add_relation(subjec, relation, objec)
    facts_sports[sport] = get_subgraph(g.nodes[sport], 0, 3)
    print('Sport %s graph number of facts: %s' % (sport, len(facts_sports[sport])))

Sport hockey graph number of facts: 7677
Sport basketball graph number of facts: 8433
Sport baseball graph number of facts: 9286
Sport tennis graph number of facts: 1413
Sport golf graph number of facts: 7658
Sport soccer graph number of facts: 1328
Sport softball graph number of facts: 2918
Sport football graph number of facts: 8576
Sport collegebaseball graph number of facts: 948
